Paste your bibtex here in UTF-8 encoding to the input.bib, and run the following code.

Note: A small number of references may still not have URLs added automatically. For these cases, please write the url manually.

Author: Hakaze Cho.

In [48]:
%pip install bibtexparser
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [39]:
import requests
import bibtexparser
import xml.etree.ElementTree as ET
import time
import os

In [ ]:
try:
    os.chdir(os.path.join(os.getcwd(), '/home/s2320415/Auto_Bib_Link')) # '.' if the path is to current folder
    #print(os.getcwd())
except:
    pass

In [41]:
with open('input.bib', 'rb') as bibtex_file:
    library = bibtexparser.load(bibtex_file)

Entry type software not standard. Not considered.


In [42]:
# Arxiv Title Search
count = 0
succeed = 0
for paper in library.entries:
    count += 1
    if 'url' not in paper:
        print(f"Missing URL for {paper['ID']}, \"{paper['title']}\". Searching for URL...")
        # search for URL
        title = paper['title']
        title_replaced = title.replace(" ", "+")
        query = f"http://export.arxiv.org/api/query?search_query=ti:{title_replaced}&sortBy=relevance&max_results=50"
        response = requests.get(query)
        root = ET.fromstring(response.content)
        entrys = root.findall('{http://www.w3.org/2005/Atom}entry')
        for entry in entrys:
            link = entry.find('{http://www.w3.org/2005/Atom}link')
            titlefound = entry.find('{http://www.w3.org/2005/Atom}title')
            if link is not None:
                if title.lower() != titlefound.text.lower().replace("\n ", ""):
                    print(f"Title found for {paper['ID']} does not match: \"{titlefound.text}\", rejected.")
                else:
                    paper['url'] = link.attrib['href']
                    succeed += 1
                    print(f"Found URL for {paper['ID']}, \"{paper['title']}\": {paper['url']}, title found: \"{titlefound.text}\", accepted.")
                    break
            else:
                print(f"Could not find URL for {paper['ID']}.")
        else:
            print(f"Could not find URL for {paper['ID']}.")
    else:
        print(f"URL already exists for {paper['ID']}, \"{paper['title']}\".")
        succeed += 1
    print("\n")
print(f"Found {succeed} URLs out of {count} papers.")


Missing URL for kossen2024context, "In-context learning learns label relationships but is not conventional learning". Searching for URL...
Title found for kossen2024context does not match: "What In-Context Learning "Learns" In-Context: Disentangling Task
  Recognition and Task Learning", rejected.
Title found for kossen2024context does not match: "Point-In-Context: Understanding Point Cloud via In-Context Learning", rejected.
Title found for kossen2024context does not match: "Skeleton-in-Context: Unified Skeleton Sequence Modeling with In-Context
  Learning", rejected.
Title found for kossen2024context does not match: "Rewards-in-Context: Multi-objective Alignment of Foundation Models with
  Dynamic Preference Adjustment", rejected.
Title found for kossen2024context does not match: "Knowledge-in-Context: Towards Knowledgeable Semi-Parametric Language
  Models", rejected.
Found URL for kossen2024context, "In-context learning learns label relationships but is not conventional learning": 

In [43]:
# Arxiv Number Search
count = 0
succeed = 0
for paper in library.entries:
    count += 1
    if 'url' not in paper:
        print(f"Missing URL for {paper['ID']}, \"{paper['title']}\". Searching for URL...")
        # search for URL
        title = paper['title']
        if 'journal' in paper:
            journal = paper['journal']
            if 'arXiv' in journal:
                arxiv_number = journal.split('arXiv:')[1].split(' ')[0]
                print(f"Arxiv number found for {paper['ID']}: {arxiv_number}.")
                paper['url'] = f"https://arxiv.org/abs/{arxiv_number}"
                print(f"Found URL for {paper['ID']}, \"{paper['title']}\": {paper['url']}, accepted.")
                succeed += 1
            else:
                print(f"Arxiv number not found for {paper['ID']}.")
        else:
            print(f"Journal not found for {paper['ID']}.")
    else:
        print(f"URL already exists for {paper['ID']}, \"{paper['title']}\".")
        succeed += 1
    print("\n")
print(f"Found {succeed} URLs out of {count} papers.")


URL already exists for kossen2024context, "In-context learning learns label relationships but is not conventional learning".


URL already exists for wang2023label, "Label Words are Anchors: An Information Flow Perspective for Understanding In-Context Learning".


Missing URL for gu2023pre, "Pre-Training to Learn in Context". Searching for URL...
Journal not found for gu2023pre.


URL already exists for han2023understanding, "Understanding In-Context Learning via Supportive Pretraining Data".


URL already exists for yoo2022ground, "Ground-Truth Labels Matter: A Deeper Look into Input-Label Demonstrations".


Missing URL for min2022rethinking, "Rethinking the Role of Demonstrations: What Makes In-Context Learning Work?". Searching for URL...
Journal not found for min2022rethinking.


Missing URL for jiang2023generative, "Generative Calibration for In-context Learning". Searching for URL...
Journal not found for jiang2023generative.


URL already exists for zhou2024batch, "Batch Calibra

In [46]:
# Semantics Scholar Search
count = 0
succeed = 0
for paper in library.entries:
    count += 1
    if 'url' not in paper:
        print(f"Missing URL for {paper['ID']}, \"{paper['title']}\". Searching for URL...")
        # search for URL
        title = paper['title']
        query = f"https://api.semanticscholar.org/graph/v1/paper/search/match?query={title}"
        while True:
            time.sleep(1)
            response = requests.get(query)
            if response.status_code == 200 or response.status_code == 404:
                break
        if response.status_code == 404:
            print(f"Could not find URL for {paper['ID']}.\n")
            continue
        response_json = response.json()
        print(response_json)
        if response_json['data'] == []:
            print(f"Could not find URL for {paper['ID']}.\n")
            continue
        else:
            paperID = response_json['data'][0]['paperId']
            query_new = f"https://api.semanticscholar.org/graph/v1/paper/{paperID}?fields=url,openAccessPdf"
            while True:
                time.sleep(1)
                response_new = requests.get(query_new)
                if response_new.status_code == 200 or response_new.status_code == 404:
                    break
            if response_new.status_code == 404:
                print(f"Could not find URL for {paper['ID']}.\n")
                continue
            response_new_json = response_new.json()
            scholar_url = response_new_json['url']
            if "openAccessPdf" in response_new_json and response_new_json['openAccessPdf'] is not None:
                paper['url'] = response_new_json['openAccessPdf']['url']
                print(f"Found URL for {paper['ID']}, \"{paper['title']}\": {paper['url']}, accepted.")
            else:
                paper['url'] = scholar_url
                print(f"Found URL for {paper['ID']}, \"{paper['title']}\": {paper['url']}, accepted.")
            succeed += 1
    else:
        print(f"URL already exists for {paper['ID']}, \"{paper['title']}\".")
        succeed += 1
    print("\n")
print(f"Found {succeed} URLs out of {count} papers.")

URL already exists for kossen2024context, "In-context learning learns label relationships but is not conventional learning".


URL already exists for wang2023label, "Label Words are Anchors: An Information Flow Perspective for Understanding In-Context Learning".


URL already exists for gu2023pre, "Pre-Training to Learn in Context".


URL already exists for han2023understanding, "Understanding In-Context Learning via Supportive Pretraining Data".


URL already exists for yoo2022ground, "Ground-Truth Labels Matter: A Deeper Look into Input-Label Demonstrations".


URL already exists for min2022rethinking, "Rethinking the Role of Demonstrations: What Makes In-Context Learning Work?".


URL already exists for jiang2023generative, "Generative Calibration for In-context Learning".


URL already exists for zhou2024batch, "Batch Calibration: Rethinking Calibration for In-Context Learning and Prompt Engineering".


URL already exists for han2023prototypical, "Prototypical calibration for few-s

In [47]:
print(f"Found {succeed} URLs out of {count} papers.")
with open('output.bib', 'w') as bibtex_file:
    bibtexparser.dump(library, bibtex_file)
print("Done. Output saved as output.bib.")

Found 73 URLs out of 76 papers.
Done. Output saved as output.bib.
